# 6. Passage Retrieval - Scaling up with FAISS

## 강의소개

- 앞선 4, 5강에서 sparse / dense embedding을 활용해 문서 검색을 하는 방법에 대해 실습해보았습니다.
- 실습에서는 수만개 단위의 데이터였지만, 실제 문서 검색이 이루어지길 원하는 실제 상황에서는 그 문서의 수가 기하급수적으로 늘어나게 됩니다.
- 위키피디아 문서에서 검색하는 상황을 가정하더라도 5백만개 이상의 문서에서 검색을 수행해야 하고, 실제로는 수천만 ~ 억개의 문서가 존재할 수 있습니다.
- 이런 상황에서는 앞선 강의에서 배운 모든 문서들에 대해 검색을 수행하는 방법이 굉장히 오랜 시간과 많은 자원을 요구하게 됩니다.

<br>

- 이번 6강에서는 이렇게 scale이 커진 상황에서 어떻게 효율적으로 검색을 수행할 수 있을지에 대해 배워볼 예정입니다.
- 먼저 similarity search의 개념에 대해 간략하게 복습해본 후, 보다 효율적인 approximate search가 무엇인지 알아보겠습니다.
- 그리고 approximate search를 편리하게 사용할 수 있도록 도와주는 FAISS를 활용하는 방법에 대해 알아보고 실습해보겠습니다.

<br>

## Further Reading

- [FAISS blog](https://engineering.fb.com/2017/03/29/data-infrastructure/faiss-a-library-for-efficient-similarity-search/)
- [FAISS github](https://github.com/facebookresearch/faiss)
- [FAISS tutorial](https://github.com/facebookresearch/faiss/tree/master/tutorial/python)
- [Getting started with Faiss](https://www.pinecone.io/learn/faiss-tutorial/)

<br>

## Reference

- [https://engineering.fb.com/2017/03/29/data-infrastructure/faiss-a-library-for-efficient-similarity-search/](https://engineering.fb.com/2017/03/29/data-infrastructure/faiss-a-library-for-efficient-similarity-search/)
- [https://github.com/facebookresearch/faiss/wiki/Indexing-1M-vectors](https://github.com/facebookresearch/faiss/wiki/Indexing-1M-vectors)
- [https://github.com/facebookresearch/faiss](https://github.com/facebookresearch/faiss)

<br>

## 6.1 Passage Retrieval and Similarity Search

### 6.1.1 복습: Retrieval with dense embedding

&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;
<img src='https://drive.google.com/uc?id=1pEnURc5hy6mUfBP42orqsk1DN9oOGAMt' width=800/>

- inference
  - passage와 query를 각각 임베딩한 후 query로부터 거리가 가까운 순서대로 passage에 순위를 매김

&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;
<img src='https://drive.google.com/uc?id=1pGXGpiUsVjAuHaoIv3UfkKaJ4xU7ViQ5' width=800/>

&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;
<img src='https://drive.google.com/uc?id=1pN7ieY0AkBZ_lVZ172OrI4S_CLYQZbvk' width=800/>

- passage의 갯수가 늘어날 수록 스코어 계산의 필요한 연산량이 중요한 이슈가 된다.
- How to find the passage in real time? -> Similarity Search!

<br>

### 6.1.2 MIPS (Maximum Inner Product Search)

- 주어진 질문(query) 벡터 q에 대해 Passage 벡터 v들 중 가장 질문과 관련된 벡터를 찾아야 한다.
  - 여기서 관련성은 내적(inner product)이 가장 큰 것

&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;
<img src='https://drive.google.com/uc?id=1pVUn_bzode15J9lm5IbT9oNxchaP7Hen' width=800/>

- 인덱싱 (indexing)
  - 방대한 양의 passage 벡터들을 저장하는 방법
- 검색 (search)
  - 인덱싱된 벡터들 중 질문 벡터와 가장 내적값이 큰 상위 k개의 벡터를 찾는 과정

<br>

- sparse embedding과 dense embedding에서 사용한 검색 방법 - brute-force(exhaustive) search
  - 저장해둔 모든 sparse/dense 임베딩에 대해 일일히 내적값을 계산하여 가장 값이 큰 passage를 추출

<br>

### 6.1.3 MIPS in Passage Retrieval

&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;
<img src='https://drive.google.com/uc?id=1pcUA_mtNCyMXjo2W7KAGQ5oOQ27s3LS4' width=800/>

- 인코딩을 한 다음 검색하는 과정을 자세히 알아보자.

<br>

### 6.1.4 MIPS & Challenges

- 실제로 검색해야 할 데이터는 훨씬 방대함
  - 5백만 개 (위키피디아)
  - 수십억, 조 단위까지 커질 수 있음
- 따라서 더 이상 모든 문서 임베딩을 일일히 보면서 검색할 수 없음

<br>

### 6.1.5 Tradeoffs of similarity search

1. Search Speed
  - 쿼리 당 유사한 벡터를 k개 찾는 데 얼마나 걸리는 지?
  - 가지고 있는 벡터량이 클수록 더 오래 걸림
2. Memory Usage
  - 벡터를 사용할 때 어디에서 가져올 것인지?
  - RAM에 모두 올려둘 수 있으면 빠르지만 많은 RAM 용량을 요구함
  - 디스크에서 계속 불러와야한다면 속도가 느려짐
3. Accuracy
  - brute-force 검색 결과와 얼마나 비슷한지?
  - 속도를 증가시키려면 정확도를 희생해야 하는 경우가 많음


<br>

- 위의 3가지 문제에 접근하는 방법론이 하나씩 존재한다.
  - Accuracy -> Exhaustive search
  - Memory -> Compression
  - Search Speed -> Pruning

&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;
<img src='https://drive.google.com/uc?id=1plG83K223a0ZpVmleCaEOm07kmXeO4Y6' width=600/>

<br>

### 6.1.6 Tradeoff of search speed and accuracy

- 속도(search time)와 재현율(recall)의 관계
  - 더 정확한 검색을 하려면 더 오랜 시간이 소모됨

&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;
<img src='https://drive.google.com/uc?id=1pvWuUINpvBFzWBw9fmcAl3ICUE-NlIj_' width=600/>

<br>

### 6.1.7 Increasing search space by bigger corpus

- 코퍼스(corpus)의 크기가 커질수록
  - 탐색 공간이 커지고 검색이 어려워짐
  - 저장해 둘 memory space 또한 많이 요구됨
  - sparse embedding의 경우 이러한 문제가 훨씬 심함
- ex) 1M docs, 500k distinct terms = 2TB (각 문서를 768차원의 벡터로 표현한 경우)

&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;
<img src='https://drive.google.com/uc?id=1pyHR1fb0YhI3mPZpfoCxqTcToetLiLvq' width=800/>

<br>

## 6.2 Approximating Similarity Search

### 6.2.1 Compression - Scalar Quantization (SQ)

- Compression
  - vector를 압축하여 하나의 vector 가 적은 용량을 차지
  - 압축량을 높일수록 메모리는 감소하고 정보 손실은 증가한다.
- ex) Scalar quantization
  - 4-byte floating point -> 1-byte (8bit) unsigned integer로 압축
  - 보통 숫자를 저장할 때는 4byte의 float32를 사용한다.
  - 실제 inner product search를 할 때는 4byte까지 필요한 경우는 많지 않다.
  - 1byte로 compress하고 inner product search를 수행해도 성능이 잘 나온다.

&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;
<img src='https://drive.google.com/uc?id=1q2NJOSpLFC03WVUEHkWVljS_zwJsbCt1' width=800/>

<br>

### 6.2.2 Pruning - Inverted File (IVF)

- Pruning
  - Search space를 줄여 search 속도 개선
  - dataset의 subset만 방문

&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;
<img src='https://drive.google.com/uc?id=1q9GDdY4qWvxTXBqJSu1GgIXlQ54g_FO6' width=400/>

<br>

#### 6.2.2.1 Clustering + Inverted file을 활용한 search

1. Clustering
  - 전체 vector space를 k개의 cluster로 나눔
  - ex) k-means clustering

&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;
<img src='https://drive.google.com/uc?id=1qBldSmGAhQqx7SOWnRhoinvYZQV1olzT' width=250/>

2. Inverted file (IVF)
  - vector의 index = inverted list structure
  - 각 cluster의 centroid id와 해당 cluster의 vector들이 연결되어 있는 형태

&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;
<img src='https://drive.google.com/uc?id=1qGFHlfZOKaT2GgebEWWMuk-AVzGkxH7Z' width=600/>

<br>

#### 6.2.2.2 Searching with clustering and IVF

1. 주어진 query vector에 대해 근접한 centroid 벡터를 찾음
2. 찾은 cluster의 inverted list 내 vector들에 대해 서치 수행

&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;
<img src='https://drive.google.com/uc?id=1qGrk14GwUUnAO1UJUZlp4davSaSTZy9j' width=800/>

<br>

## 6.3 Introduction to FAISS

### 6.3.1 What is FAISS

- FAISS github
  - [https://github.com/facebookresearch/faiss](https://github.com/facebookresearch/faiss)
- FAISS tutorial
  - [https://github.com/facebookresearch/faiss/tree/master/tutorial/python](https://github.com/facebookresearch/faiss/tree/master/tutorial/python)

&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;
<img src='https://drive.google.com/uc?id=1qPL4Kdyf60jKExfkaPeAJwbO88dzv7k_' width=800/>

<br>

- FAISS: Library for efficient similarity search

&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;
<img src='https://drive.google.com/uc?id=1qSJM1utKSVBlrdMnvKOYFkmcpoJouGTQ' width=800/>

<br>

### 6.3.2 Passage Retrieval with FAISS

#### 6.3.2.1 Train index and map vectors

&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;
<img src='https://drive.google.com/uc?id=1qT2cWSzVpLM5S-_FbB52SR_tvp5bfNmB' width=800/>

- 먼저 벡터들을 학습한다.
  - FAISS를 활용하려면 먼저 cluster를 확보해야 한다.
  - 이 클러스터들을 데이터 포인트의 분포를 보고 적절하게 지정하기 위해 학습 데이터가 필요하다.
  - compress를 위해서도 학습이 필요하다.

<br>

#### 6.3.2.2 Search based on FAISS index

- nprobe
  - 몇 개의 가장 가까운 cluster를 방문하여 search 할 것인지

&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;
<img src='https://drive.google.com/uc?id=1qaR0B0Iv5bK6y08p7CqlSx-SEQ6q8YOO' width=800/>

<br>

## 6.4 Scaling up with FAISS

### 6.4.1 FAISS Basics

- brute-force로 모든 벡터와 쿼리를 비교하는 가장 단순한 인덱스 만들기

&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;
<img src='https://drive.google.com/uc?id=1qdw-fMRimR7ZfPcJciMkcNxS1AmF9gOv' width=800/>

<br>

### 6.4.2 IVF with FAISS

- IVF 인덱스 만들기
- 클러스터링을 통해서 가까운 클러스터 내 벡터들만 비교함
- 빠른 검색 가능
- 클러스터 내에서는 여전히 전체 벡터와 거리 비교 (Flat)

&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;
<img src='https://drive.google.com/uc?id=1qesX9DZRbOzSYprYnF9WEV_C_7HjJ4c3' width=800/>

- `quantizer`
  - 클러스터에서 거리를 어떻게 측정할지에 대한 방법
  - 거리 측정 방식인 L2를 사용하기 때문에 `IndexFlatL2()`를 사용한다.
- `IndexIVFFlat()`
  -  클러스터 생성
  - `nlist`: 클러스터 갯수
- `index.train(xb)`
  - `xb`를 통해 K-Means를 이용하여 클러스터들을 생성한다.

<br>

### 6.4.3 IVF-PQ with FAISS

- 벡터 압축 기법(PQ) 활용하기
- 전체 벡터를 저장하지 않고 압축된 벡터만을 저장
- 메모리 사용량을 줄일 수 있음

&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;
<img src='https://drive.google.com/uc?id=1qf2DQ3ac3-54U_jok32Oq5vzM3HUnsfK' width=800/>

<br>

### 6.4.4 Using GPU with FAISS

- GPU의 빠른 연산 속도를 활용할 수 있음
  - 거리 계산을 위한 행렬곱 등에서 유리
- 다만, GPU 메모리 제한이나 메모리 random access 시간이 느린 것 등이 단점

&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;
<img src='https://drive.google.com/uc?id=1qoGn-DC2wBYpXbn0JdQkZGRKHqsT0y12' width=800/>

<br>

### 6.4.5 Using Multiple GPUs with FAISS

- 여러 GPU를 활용하여 연산 속도를 한층 더 높일 수 있음

&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;
<img src='https://drive.google.com/uc?id=1qp2-CMxUei_D7LOzE2DW6ga_xSmo4kZ5' width=800/>